In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import joblib

# 1. Load dataset
df = pd.read_csv(r"D:\downloads\physico_dataset_1500.csv")  # change path if needed

print("Dataset shape:", df.shape)
print(df.head())

# 2. Features and target
X = df[["knee_angle", "hip_angle", "elbow_angle", "symmetry"]]
y = df["risk_label"]

# 3. Encode target labels (Low, Medium, High -> 0,1,2)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

print("Label mapping:", dict(zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))))

# 4. Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 5. Feature scaling (optional but good practice)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 6. Train model
model = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    max_depth=None,
    min_samples_split=2
)

model.fit(X_train_scaled, y_train)

# 7. Predictions
y_pred = model.predict(X_test_scaled)

# 8. Evaluation
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred, target_names=label_encoder.classes_))

# 9. Save model and preprocessors
joblib.dump(model, "physico_risk_model.pkl")
joblib.dump(scaler, "scaler.pkl")
joblib.dump(label_encoder, "label_encoder.pkl")

print("\nModel and preprocessors saved successfully.")


Dataset shape: (1500, 5)
   knee_angle  hip_angle  elbow_angle  symmetry risk_label
0      127.45     109.34        70.92     0.995        Low
1      117.93      93.39        81.40     0.788       High
2      129.72      90.18        85.86     0.788       High
3      142.85      99.96       108.88     0.806        Low
4      116.49      97.96        95.57     0.869        Low
Label mapping: {'High': np.int64(0), 'Low': np.int64(1), 'Medium': np.int64(2)}

Accuracy: 0.9933333333333333

Confusion Matrix:
 [[112   0   0]
 [  0 102   2]
 [  0   0  84]]

Classification Report:
               precision    recall  f1-score   support

        High       1.00      1.00      1.00       112
         Low       1.00      0.98      0.99       104
      Medium       0.98      1.00      0.99        84

    accuracy                           0.99       300
   macro avg       0.99      0.99      0.99       300
weighted avg       0.99      0.99      0.99       300


Model and preprocessors saved successf

In [6]:
import joblib
import numpy as np

model = joblib.load("physico_risk_model.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("label_encoder.pkl")

# Example new data: knee, hip, elbow, symmetry
new_data = np.array([[150, 920, 880, 0.82]])

new_data_scaled = scaler.transform(new_data)
prediction = model.predict(new_data_scaled)
risk_label = label_encoder.inverse_transform(prediction)

print("Predicted Risk:", risk_label[0])


Predicted Risk: Low


d:\ai_physicothe\env\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [8]:
import cv2
import numpy as np
import joblib
import math
import mediapipe as mp
from mediapipe.tasks import python
from mediapipe.tasks.python import vision

# Load ML artifacts
model = joblib.load("physico_risk_model.pkl")
scaler = joblib.load("scaler.pkl")
label_encoder = joblib.load("label_encoder.pkl")

# Load MediaPipe Pose Landmarker Task
base_options = python.BaseOptions(model_asset_path=r"D:\ai_physicothe\pose_landmarker_lite.task")
options = vision.PoseLandmarkerOptions(
    base_options=base_options,
    running_mode=vision.RunningMode.VIDEO,
    num_poses=1,
    min_pose_detection_confidence=0.5,
    min_pose_presence_confidence=0.5,
    min_tracking_confidence=0.5
)
landmarker = vision.PoseLandmarker.create_from_options(options)

def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc) + 1e-6)
    angle = np.degrees(np.arccos(np.clip(cosine_angle, -1.0, 1.0)))
    return angle

def get_symmetry(left_val, right_val):
    if left_val + right_val == 0:
        return 1.0
    return 1 - abs(left_val - right_val) / (left_val + right_val)

cap = cv2.VideoCapture(0)
timestamp = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)

    result = landmarker.detect_for_video(mp_image, timestamp)
    timestamp += 1

    if result.pose_landmarks:
        lm = result.pose_landmarks[0]

        def pt(i):
            return [lm[i].x, lm[i].y]

        # Joint points
        left_hip, left_knee, left_ankle = pt(23), pt(25), pt(27)
        right_hip, right_knee, right_ankle = pt(24), pt(26), pt(28)

        left_shoulder, left_elbow, left_wrist = pt(11), pt(13), pt(15)
        right_shoulder, right_elbow, right_wrist = pt(12), pt(14), pt(16)

        # Angles
        left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
        right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
        knee_angle = (left_knee_angle + right_knee_angle) / 2

        left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
        right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
        hip_angle = (left_hip_angle + right_hip_angle) / 2

        left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
        right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)
        elbow_angle = (left_elbow_angle + right_elbow_angle) / 2

        # Symmetry
        symmetry_knee = get_symmetry(left_knee_angle, right_knee_angle)
        symmetry_hip = get_symmetry(left_hip_angle, right_hip_angle)
        symmetry_elbow = get_symmetry(left_elbow_angle, right_elbow_angle)
        symmetry = np.mean([symmetry_knee, symmetry_hip, symmetry_elbow])

        # ML Prediction
        X = np.array([[knee_angle, hip_angle, elbow_angle, symmetry]])
        X_scaled = scaler.transform(X)
        pred = model.predict(X_scaled)
        risk_label = label_encoder.inverse_transform(pred)[0]

        # Overlay
        cv2.putText(frame, f"Knee: {int(knee_angle)}", (20, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.putText(frame, f"Hip: {int(hip_angle)}", (20, 70),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.putText(frame, f"Elbow: {int(elbow_angle)}", (20, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.putText(frame, f"Symmetry: {symmetry:.2f}", (20, 130),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)
        cv2.putText(frame, f"Risk: {risk_label}", (20, 170),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0,255,255), 2)

    cv2.imshow("Physico Risk Prediction (MediaPipe Tasks)", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


d:\ai_physicothe\env\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
d:\ai_physicothe\env\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
d:\ai_physicothe\env\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
d:\ai_physicothe\env\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
d:\ai_physicothe\env\Lib\site-packages\sklearn\utils\validation.py:2691: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
d:\ai_physicothe\env\Lib\site-packages\sklearn\utils\va